In [1]:
import data_extraction
import pandas as pd
import yaml, boto3

pd.set_option('display.max_columns', None)

myObj = data_extraction.DataExtractor()

s3 = boto3.client('s3')

s3.download_file("data-handling-public",'products.csv','products.csv')



In [2]:
df = pd.read_csv("products.csv")

df is a dataframe that now contains the data from our table
These are the first & last few lines of the database then!

In [3]:
df.head(5)

,Unnamed: 0,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a


In [17]:
df.tail(5)

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
1848,Goodmans Qi Autosense In Car Phone Holder,15.0,0.125,diy,8185827751600,2020-08-28,3ff21cb1-8e90-499d-b5a0-6839fb83e70f,Still_avaliable,x4-2227164k
1849,Goodmans Qi Wireless Charging Phone Stand,15.0,0.134,diy,9769724533935,2012-05-20,a7910f74-c2b6-4144-98b7-30ca68f34d2d,Still_avaliable,i3-8338545Z
1850,RAC 12V Wet & Dry Vacuum,18.0,0.820,diy,3113855860327,1993-11-28,66d3efa0-f480-4da9-893b-e8aaff53e27b,Still_avaliable,n4-1642658b
1851,Goodmans Qi Autosense In Car Phone Holder,15.0,0.125,diy,9195087467400,2016-09-20,efb472f1-8f00-448e-a874-9fa9812eeab5,Still_avaliable,A3-7619070S
1852,Beldray Telescopic Ladder 2.9m,69.0,9.400,diy,8518402353072,2006-10-07,cfe701da-6493-4a7f-83cf-9aab1650f689,Still_avaliable,i6-2851854w


try to clean up thei weights

In [4]:
import data_cleaning

myObj = data_cleaning.DataCleaning()

df = myObj.convert_product_weights(df=df)

could not convert string to float: '9GO9NZ5JT'
could not convert string to float: 'Z8ZTDGUZV'
could not convert string to float: 'MX180RYSH'
could not convert string to float: '77g '
could not convert string to float: '16o'


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1853 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_name   1849 non-null   object        
 1   product_price  1846 non-null   float64       
 2   weight         1844 non-null   float64       
 3   category       1849 non-null   category      
 4   EAN            1849 non-null   object        
 5   date_added     1846 non-null   datetime64[ns]
 6   uuid           1849 non-null   object        
 7   removed        1849 non-null   category      
 8   product_code   1849 non-null   object        
dtypes: category(2), datetime64[ns](1), float64(2), object(4)
memory usage: 105.7+ KB


Lets drop the unlabelled column

In [7]:
df = df.drop(columns=["Unnamed: 0"])

remove £ from price

In [10]:
df['product_price'] = df['product_price'].str.replace('£', '', regex=False)
df['product_price'] = pd.to_numeric(df['product_price'],errors="coerce")

We have some date columns

In [11]:
df['date_added']=pd.to_datetime(df['date_added'],infer_datetime_format=True, errors='coerce')

inspect some potentially categorical data:
removed
category

In [13]:
df["removed"].describe()

count                1849
unique                  5
top       Still_avaliable
freq                 1752
Name: removed, dtype: object

In [14]:
df["category"].describe()

count         1849
unique          10
top       homeware
freq          1138
Name: category, dtype: object

Not bad.

In [15]:
df["removed"] = df["removed"].astype("category")
df["category"] = df["category"].astype("category")

Yes. What about duplicate lines?

In [19]:
df.duplicated().sum()

3

3! ok. Let's just check on the critical columns only - not system generated ones

In [20]:
df.duplicated(subset=["product_name", "EAN"],keep="first").sum()

3

Still 3
lets drop nas too

In [22]:
df.drop_duplicates(subset=["product_name", "EAN"],keep="first",inplace=True)
df.dropna(inplace=True,axis="index",how="any") # drops the whole row if any columns have an nan


make sure we've not dropped everythign

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1844 entries, 0 to 1852
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_name   1844 non-null   object        
 1   product_price  1844 non-null   float64       
 2   weight         1844 non-null   float64       
 3   category       1844 non-null   category      
 4   EAN            1844 non-null   object        
 5   date_added     1844 non-null   datetime64[ns]
 6   uuid           1844 non-null   object        
 7   removed        1844 non-null   category      
 8   product_code   1844 non-null   object        
dtypes: category(2), datetime64[ns](1), float64(2), object(4)
memory usage: 119.4+ KB


Now put the code into a local database maybe

In [24]:
import database_utils


myObject = database_utils.DatabaseConnector()


myObject.upload_to_db(df,"dim_products")

ModuleNotFoundError: No module named 'sqlalchemy'

Ideas for future:
Misspellings: https://medium.com/geekculture/how-to-fix-misspelled-words-for-your-next-nlp-project-with-one-line-of-code-98888a5ff207
Common variants of companies - "ltd." vs "ltd" vs "limited"